# Mass-spring-damper system estimation via Kalman Filter

![mass-spring-damper](Mass_spring_damper.svg.png)

In [84]:
import symforce.symbolic as sf
import matplotlib.pyplot as plt
import numpy as np

from symforce.notebook_util import display
# %matplotlib inline

In [85]:
# Governing Equation
# m * a + c * v + k * x = f
# v = dx/dt;  a = dv/dt

# STATE 
# q = [x]
#     [v]

# STATE DERIVATIVE
# qdot = [v]
#        [a]

# PLUG THINGS IN
# qdot = [q1],
#         [a]

# a = (f - k * x - c * v) / m
# a = (f - k * q0 - c * q1)/ m

# qdot = [q1]
#        [f - k*q0 - c*q1]

# LINEAR SYSTEM:
# qdot =  [ 1    0]   @  [q0]  +  [ 0 ] @ f
#         [-k/m -c/m]    [q1]     [1/m]

# Or... we do this with symforce

In [86]:
x, v, a, f, m, k, c = sf.symbols('x v a f m k c')

q = sf.V2(x, v)
qdot = sf.V2(v,a)
qdot = qdot.subs(a, (f - k * x - c * v) / m)

A = qdot.jacobian(q)
B = qdot.jacobian(f)

print("A = ")
display(A)

print("B = ")
display(B)


A = 


⎡ 0    1 ⎤
⎢        ⎥
⎢-k   -c ⎥
⎢───  ───⎥
⎣ m    m ⎦

B = 


⎡0⎤
⎢ ⎥
⎢1⎥
⎢─⎥
⎣m⎦

# First Task:
We want to create a simulation of the mass-spring-damper system: <br>
Complete the following code:



In [87]:
#INITALIZE
m = 1.0
k = 10.0
c = 1.0

t_0 = 0
dt = 0.01
t_final = 10

q = np.array([1,1]).reshape(-1,1)
A = np.array([[0, 1],
             [-k/m, -c/m]])
B = np.array([[0],
             [1/m]])
f = 1.0
t_vec =  np.arange(t_0, t_final, dt)
x_history = []

#SIMULATION
for t in t_vec:
    #------------YOUR CODE HERE------------------#
    #compute qdot, q (you can use euler integration)
    qdot = A @ q + B * f
    q = q + qdot * dt
    #------------YOUR CODE HERE------------------#
    
    x_history.append(q[0])

#PLOT
plt.figure()
plt.plot(t_vec, x_history)
plt.show()

# We want to add noisy position measurement and noisy force
h(x) ~ Normal (mu = x, std = 0.05) </br>
f ~ Normal (mu = 1, std = 3) </br>
</br>

you can use: </br> 
np.random.normal(mu,std)

In [88]:
#INITALIZE
m = 1.0
k = 10.0
c = 1.0

t_0 = 0
dt = 0.01
t_final = 10

A = np.array([[0, 1],
             [-k/m, -c/m]])
B = np.array([[0],
             [1/m]])
t_vec =  np.arange(t_0, t_final, dt)

q_gt = np.array([1,1]).reshape(-1,1)
q_est = np.array([1,1]).reshape(-1,1)

x_gt_history = []
x_est_history = []
z_history = []
#SIMULATION
for t in t_vec:
    # compute qdot_gt, q_gt, z
    f_est = 1.0
    f_real = np.random.normal(1.0,3)
    
    #------------YOUR CODE HERE------------------#
    #compute qdot_gt, q_gt, qdot_est, q_est and z

    qdot_gt = A @ q_gt + B * f_real
    q_gt = q_gt + qdot_gt * dt
    qdot_est = A @ q_est + B * f_est
    q_est = q_est + qdot_est * dt
    z = np.random.normal(q_gt[0],0.05)
    #------------YOUR CODE HERE------------------#

    x_gt_history.append(q_gt[0])
    x_est_history.append(q_est[0])
    z_history.append(z)

#PLOT
plt.figure()
plt.plot(t_vec, x_gt_history, color = 'blue')
plt.plot(t_vec, x_est_history, color = 'black')
plt.scatter(t_vec, z_history, s = 1, color = 'red')
plt.legend(['x_gt', 'x_est(predict)', 'z'])
plt.show()

# Lets add the Kalman filter to improve Estimation on x

In [89]:
#INITALIZE
m = 1.0
k = 10.0
c = 1.0

t_0 = 0
DT = 0.01
t_final = 10

A_C = np.array([[0, 1],
             [-k/m, -c/m]]) 

B_C = np.array([[0],
             [1/m]])

#c2d #https://en.wikipedia.org/wiki/Discretization
I = np.eye(2)
A_D = (I + A_C * DT)
B_D = np.linalg.inv(A_C) @ (A_D - I) @ B_C

H = np.array([[1.0, 0]])
COV_R = np.diag([0.5*(3/m)*DT**2, 0.5*(3/m)*DT])
COV_Q = np.array([[0.05**2]])
COV_PRIOR = np.eye(2) * 1e-15

t_vec =  np.arange(t_0, t_final, DT)

q_gt = np.array([1,1]).reshape(-1,1)
q_est_mu = np.array([1,1]).reshape(-1,1)
q_est_cov = COV_PRIOR

x_gt_history = []
x_est_history = []
z_history = []
x_est_std_history = []
K_history = []

def kalman(q_prev, cov_prev, f, z):
    #------------YOUR CODE HERE------------------#
    #compute mu, cov, K
    #You can use Probablistic Robotics page 36

    #predict
    mu_bar = A_D @ q_prev + B_D * f
    cov_bar = A_D @ cov_prev @ A_D.T + COV_R
    
    #update
    K = cov_bar @ H.T @ np.linalg.inv(H @ cov_bar @ H.T + COV_Q)
    mu = mu_bar + K @ (z - H @ mu_bar)
    cov = (I - K @ H) @ cov_bar
    #------------YOUR CODE HERE------------------#
    return mu, cov, K

f_est = 1.0
#SIMULATION
np.random.seed(1)
for t in t_vec:
    f_real = np.random.normal(f_est,3)
    q_gt = A_D @ q_gt + B_D * f_real
    z = np.random.normal(q_gt[0],0.05)

    q_est_mu, q_est_cov, K = kalman(q_est_mu, q_est_cov ,f_est ,z)

    x_gt_history.append(q_gt[0])
    x_est_history.append(q_est_mu[0])
    x_est_std_history.append(np.sqrt(q_est_cov[0,0]))
    z_history.append(z)
    K_history.append(K)

#PLOT
x_est_history = np.array(x_est_history).squeeze()
x_est_std_history = np.array(x_est_std_history)

# %matplotlib inline
%matplotlib auto

plt.figure()
graphics_est, = plt.plot(t_vec, x_est_history, color = 'black')
# plt.plot(t_vec, x_est_history + 1 *x_est_std_history, color = 'black')
# plt.plot(t_vec, x_est_history - 1 *x_est_std_history, color = 'black')
graphics_z = plt.scatter(t_vec, z_history, s = 1, color = 'red')
graphics_gt, = plt.plot(t_vec, x_gt_history, color = 'blue')
plt.legend([graphics_gt, graphics_est, graphics_z], ['x_gt', 'x_est', 'z'])
plt.title('kalman filter')
plt.show()


Using matplotlib backend: TkAgg


## Plot K vs time

In [90]:

plt.plot(t_vec,[K[0,0] for K in K_history], color = 'green')